In [3]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

  Cloning https://github.com/lucasb-eyer/pydensecrf.git to c:\users\daanish mittal\appdata\local\temp\pip-req-build-r0bpwsng
  Resolved https://github.com/lucasb-eyer/pydensecrf.git to commit 2723c7fa4f2ead16ae1ce3d8afe977724bb8f87f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pydensecrf


  Running command git clone --filter=blob:none --quiet https://github.com/lucasb-eyer/pydensecrf.git 'C:\Users\Daanish Mittal\AppData\Local\Temp\pip-req-build-r0bpwsng'
  error: subprocess-exited-with-error
  
  exit code: 1
  
  [11 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pydensecrf
  copying pydensecrf\utils.py -> build\lib.win-amd64-cpython-310\pydensecrf
  copying pydensecrf\__init__.py -> build\lib.win-amd64-cpython-310\pydensecrf
  running build_ext
  building 'pydensecrf.eigen' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pydensecrf
ERROR: Could not build wheels for pydensecrf, whic

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97  # Set a custom testing threshold
    return DefaultPredictor(cfg)

def mask_sides(image, mask_percentage=0.2):
    h, w = image.shape[:2]
    mask_width = int(w * mask_percentage)
    
    # Create a mask with 1s in the center and 0s on the sides
    mask = np.ones(image.shape[:2], dtype=np.uint8)
    mask[:, :mask_width] = 0
    mask[:, -mask_width:] = 0
    
    # Apply the mask to the image
    masked_image = image.copy()
    masked_image[:, :mask_width] = 0
    masked_image[:, -mask_width:] = 0
    
    return masked_image, mask

def point_line_distance(point, line_start, line_end):
    """Calculate the distance from a point to a line segment."""
    line_vec = np.array(line_end) - np.array(line_start)
    point_vec = np.array(point) - np.array(line_start)
    line_len = np.linalg.norm(line_vec)
    line_unitvec = line_vec / line_len
    point_vec_scaled = point_vec / line_len
    t = np.dot(line_unitvec, point_vec_scaled)
    t = np.clip(t, 0, 1)
    nearest = line_start + t * line_vec
    dist = np.linalg.norm(point - nearest)
    return dist

def calculate_polygon_angle(contour):
    rect = cv2.minAreaRect(contour)
    angle = rect[2]
    if angle < -45:
        angle = 90 + angle
    elif angle > 45:
        angle = angle - 90
    else:
        angle = 90 - abs(angle)
    return angle

def process_and_show_frames(predictor, metadata, video_path, num_frames=5):
    ramp_index = metadata.thing_classes.index('ramp')
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened() and frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Mask 20% from left and right sides
        masked_frame, frame_mask = mask_sides(frame, 0.3)
        
        outputs = predictor(masked_frame)
        instances = outputs["instances"].to("cpu")
        ramp_instances = instances[instances.pred_classes == ramp_index]
        
        # Create a copy of the frame to draw on
        draw_frame = masked_frame.copy()
        
        for i in range(len(ramp_instances)):
            if ramp_instances.has("pred_masks"):
                mask = ramp_instances.pred_masks[i].numpy()
                # Apply frame mask to the predicted mask
                mask = cv2.bitwise_and(mask.astype(np.uint8), frame_mask)
                
                # Find contours of the mask
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    # Find the largest contour
                    largest_contour = max(contours, key=cv2.contourArea)
                    
                    # Simplify the contour to a polygon with 4 sides
                    epsilon = 0.05 * cv2.arcLength(largest_contour, True)
                    approx_contour = cv2.approxPolyDP(largest_contour, epsilon, True)
                    
                    # Ensure the contour has 4 sides
                    if len(approx_contour) > 4:
                        approx_contour = cv2.convexHull(approx_contour)
                        approx_contour = cv2.approxPolyDP(approx_contour, epsilon, True)
                    
                    if len(approx_contour) > 4:
                        approx_contour = cv2.approxPolyDP(approx_contour, epsilon * 2, True)
                    
                    if len(approx_contour) > 4:
                        approx_contour = cv2.convexHull(approx_contour, returnPoints=False)
                        approx_contour = cv2.approxPolyDP(approx_contour, epsilon * 3, True)
                    
                    approx_contour = approx_contour[:4]

                    # Draw the simplified contour (purple polygon)
                    cv2.drawContours(draw_frame, [approx_contour], 0, (128, 0, 128), 2)
                    
                    # Draw the bounding box
                    x, y, w, h = cv2.boundingRect(largest_contour)
                    cv2.rectangle(draw_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    
                    # Extend the left and right sides of the bounding box vertically
                    extension_length = 50  # Adjust this length as needed
                    extended_edge_top_start = (x, y)
                    extended_edge_bottom_start = (x, y + h)
                    extended_edge_top_end = (x, y - extension_length)
                    extended_edge_bottom_end = (x, y + h + extension_length)
                    
                    # Draw the original vertical edges
                    cv2.line(draw_frame, extended_edge_top_start, extended_edge_bottom_start, (0, 0, 255), 2)
                    
                    # Draw the extended vertical edges (red lines)
                    cv2.line(draw_frame, extended_edge_top_start, extended_edge_top_end, (0, 0, 255), 2)
                    cv2.line(draw_frame, extended_edge_bottom_start, extended_edge_bottom_end, (0, 0, 255), 2)
                    
                    # Calculate angle between extended vertical edge and almost vertical edge of ramp
                    ramp_edge_start = tuple(approx_contour[0][0])  # First point of approximated contour
                    ramp_edge_end = tuple(approx_contour[1][0])  # Second point of approximated contour
                    
                    # Calculate the angle between these edges
                    vector_extended = np.array([extended_edge_bottom_end[0] - extended_edge_top_end[0], extended_edge_bottom_end[1] - extended_edge_top_end[1]])
                    vector_ramp = np.array([ramp_edge_end[0] - ramp_edge_start[0], ramp_edge_end[1] - ramp_edge_start[1]])
                    
                    dot_product = np.dot(vector_extended, vector_ramp)
                    norm_extended = np.linalg.norm(vector_extended)
                    norm_ramp = np.linalg.norm(vector_ramp)
                    
                    cosine_angle = dot_product / (norm_extended * norm_ramp)
                    angle = np.arccos(cosine_angle)
                    angle_degrees = np.degrees(angle)
                    
                    # Put the angle on the image
                    cv2.putText(draw_frame, f"Angle: {angle_degrees:.2f}", (x, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
                    
                    # Calculate midpoint of the screen
                    mid_x = draw_frame.shape[1] // 2
                    
                    # Find intersection point
                    intersection_point = (mid_x, y + h)
                    
                    # Calculate end point of rotated line
                    line_length = 200  # Adjust as needed
                    end_x = int(intersection_point[0] + line_length * np.sin(np.radians(angle_degrees)))
                    end_y = int(intersection_point[1] - line_length * np.cos(np.radians(angle_degrees)))
                    
                    # Draw the rotated green line
                    cv2.line(draw_frame, intersection_point, (end_x, end_y), (0, 255, 0), 2)

                    # Calculate distance between the polygon edge and the green line
                    min_distance = float('inf')
                    for point in approx_contour:
                        dist = point_line_distance(point[0], intersection_point, (end_x, end_y))
                        if dist < min_distance:
                            min_distance = dist
                    
                    # Determine tint color based on distance
                    if min_distance < 5:
                        tint_color = (0, 255, 0)  # Green
                    elif min_distance < 20:
                        tint_color = (0, 165, 255)  # Orange
                    else:
                        tint_color = (0, 0, 255)  # Red
                    
                    # Create a tinted overlay
                    overlay = np.full(draw_frame.shape, tint_color, dtype=np.uint8)
                    alpha = 0.3  # Transparency factor
                    
                    # Blend the overlay with the original frame
                    draw_frame = cv2.addWeighted(overlay, alpha, draw_frame, 1 - alpha, 0)

                    # Draw the colored line based on the distance
                    cv2.line(draw_frame, intersection_point, (end_x, end_y), tint_color, 2)
        
        plt.figure(figsize=(20, 16))
        plt.imshow(cv2.cvtColor(draw_frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Frame {frame_count}")
        plt.show()
        frame_count += 1
    
    cap.release()

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/army_ramp/loader.ramp/mask_rcnn_R_101_FPN_3x/2024-07-09-01-55-42/model_final.pth"
video_path = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test_army/IMG_1226 2.MOV"

predictor = load_model(config_file, weights_file)
metadata = MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0])

process_and_show_frames(predictor, metadata, video_path, num_frames=5)


AttributeError: Attribute 'thing_classes' does not exist in the metadata of dataset 'loader.ramp-train': metadata is empty.